In [1]:
# BLOQUE 1: Instalación de Librerías
# ---
# Instala las librerías que no vienen por defecto en Colab y son
# necesarias para el pipeline guardado.
# Solo necesitas ejecutar esto una vez por sesión (o si el entorno se reinicia).

!pip install category_encoders==2.6.3 geopy -q
# Nota: Se especifica version de category_encoders por compatibilidad, ajustar si da problemas.
#       (La versión usada en el notebook original puede variar, si usaste una diferente
#        en el entrenamiento, instálala aquí). XGBoost y Scikit-learn suelen estar
#        preinstalados en Colab o ser compatibles.

In [2]:
# BLOQUE 2: Importaciones y Montar Google Drive
# ---
# Importa todas las librerías necesarias y monta tu Google Drive
# para acceder al archivo del pipeline guardado.

import pandas as pd
import numpy as np
import joblib
import os
from google.colab import drive
from geopy.distance import geodesic

# Importar clases específicas es necesario para que joblib pueda deserializar los objetos
import category_encoders as ce
from sklearn.preprocessing import StandardScaler, SplineTransformer, PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import xgboost as xgb # Aunque no lo usemos directamente, es necesario para cargar el modelo

# Montar Google Drive
drive.mount('/content/drive')
print("Google Drive montado.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado.


In [3]:
# BLOQUE 3: Cargar el Pipeline Guardado
# ---
# Carga el diccionario completo que contiene todos los componentes
# del pipeline desde el archivo .joblib.

# Asegúrate de que esta ruta coincida EXACTAMENTE con donde guardaste el archivo
pipeline_file_path = '/content/drive/MyDrive/ModelosIdealista/pipeline_idealista_completo.joblib' # cite: 1

if os.path.exists(pipeline_file_path):
    pipeline_components = joblib.load(pipeline_file_path) # cite: 1
    print(f"Pipeline cargado exitosamente desde: {pipeline_file_path}")
    print(f"Componentes cargados: {list(pipeline_components.keys())}") # cite: 1
else:
    print(f"ERROR: No se encontró el archivo del pipeline en {pipeline_file_path}")
    pipeline_components = None # Poner None si no se carga

Pipeline cargado exitosamente desde: /content/drive/MyDrive/ModelosIdealista/pipeline_idealista_completo.joblib
Componentes cargados: ['scaler_geo', 'kmeans_geo', 'pca_geo', 'target_encoder', 'spline_transformer', 'poly_features', 'scaler_final', 'xgb_model', 'pois', 'floor_map', 'imputation_values', 'cluster_avg_logprice_map', 'spline_input_cols', 'poly_base_cols', 'te_cols', 'scaled_num_cols', 'feature_order']


In [4]:
# BLOQUE 4: Definir Datos de Entrada para una Nueva Vivienda
# ---
# Crea un diccionario con los datos de la vivienda para la que quieres
# predecir el precio. Usa los nombres de columna originales ANTES de
# cualquier transformación (como se cargaron en el Bloque 2 del notebook original).
# Sustituye los valores de ejemplo por los de tu vivienda.

datos_nueva_vivienda = {
    # --- Características Principales ---
    'size': 85,              # Metros cuadrados
    'rooms': 3,              # Número de habitaciones
    'bathrooms': 2,          # Número de baños
    'floor': '2',            # Planta (ej. 'bj', '1', '2', 'atico', etc.) - el pipeline lo manejará
    'hasLift': 1,            # 1 si tiene ascensor, 0 si no
    'exterior': True,        # True si es exterior, False si es interior, 'Unknown' si no se sabe
    'propertyType': 'flat',  # Tipo de propiedad (ej. 'flat', 'penthouse', 'duplex')
    'status': 'good',        # Estado (ej. 'good', 'newdevelopment', 'renew')
    'numPhotos': 15,         # Número de fotos en el anuncio (o un valor razonable)

    # --- Ubicación ---
    'latitude': 41.3851,     # Latitud (ej. cerca de Plaça Catalunya)
    'longitude': 2.1734,     # Longitud (ej. cerca de Plaça Catalunya)

    # --- Parking ---
    'hasParking': 0,         # 1 si tiene parking (aunque sea opcional), 0 si no
    'isParkingIncludedInPrice': 0 # 1 si el parking (si existe) está incluido, 0 si no

    # --- Columnas 'isna_' ---
    # Estas se crearán automáticamente en la función de predicción si algún
    # valor original de las columnas clave es NaN. Para una entrada manual,
    # normalmente no las incluirías aquí a menos que simules un NaN original.
    # 'isna_size': 0,
    # 'isna_rooms': 0,
    # ... etc.
}

# Convertir el diccionario a un DataFrame de Pandas (con una sola fila)
input_df = pd.DataFrame([datos_nueva_vivienda])

print("DataFrame de entrada creado:")
print(input_df.to_markdown(index=False)) # Usar to_markdown para mejor visualización en Colab

DataFrame de entrada creado:
|   size |   rooms |   bathrooms |   floor |   hasLift | exterior   | propertyType   | status   |   numPhotos |   latitude |   longitude |   hasParking |   isParkingIncludedInPrice |
|-------:|--------:|------------:|--------:|----------:|:-----------|:---------------|:---------|------------:|-----------:|------------:|-------------:|---------------------------:|
|     85 |       3 |           2 |       2 |         1 | True       | flat           | good     |          15 |    41.3851 |      2.1734 |            0 |                          0 |


In [10]:
# BLOQUE 5: Definir la Función de Predicción (LIMPIA Y FINAL)
# ---

def predict_price(input_data_df, pipeline_components):
    """
    Realiza la predicción de precio para nuevos datos usando el pipeline cargado.
    (Resto de la docstring igual)
    """
    # Importaciones locales
    import numpy as np
    import pandas as pd
    from geopy.distance import geodesic

    if pipeline_components is None:
        print("Error: El pipeline no está cargado.")
        return None

    # Crear una copia
    df_processed = input_data_df.copy()

    # Extraer componentes
    floor_map = pipeline_components['floor_map']
    imputation_values = pipeline_components['imputation_values']
    pois = pipeline_components['pois']
    scaler_geo = pipeline_components['scaler_geo']
    kmeans_geo = pipeline_components['kmeans_geo']
    pca_geo = pipeline_components['pca_geo']
    cluster_avg_logprice_map = pipeline_components['cluster_avg_logprice_map']
    te = pipeline_components['target_encoder']
    te_cols = pipeline_components['te_cols']
    spline = pipeline_components['spline_transformer']
    spline_input_cols = pipeline_components['spline_input_cols']
    poly = pipeline_components['poly_features']
    poly_base_cols = pipeline_components['poly_base_cols']
    scaler_final = pipeline_components['scaler_final']
    scaled_num_cols = pipeline_components['scaled_num_cols']
    feature_order = pipeline_components['feature_order']
    model = pipeline_components['xgb_model']

    print("\n--- Iniciando Preprocesamiento ---")

    # --- 1. Limpieza, Formateo e Imputación ---
    print("Aplicando limpieza e imputación...")
    # (Código igual que antes...)
    median_floor = imputation_values['median_floor']
    df_processed['floor'] = pd.to_numeric(df_processed['floor'].replace(floor_map), errors='coerce').fillna(median_floor)
    mode_exterior = imputation_values['mode_exterior']
    df_processed['exterior'] = pd.to_numeric(df_processed['exterior'].replace({'Unknown': np.nan, True: 1, False: 0}), errors='coerce').fillna(mode_exterior).astype(int)
    mode_hasLift = imputation_values['mode_hasLift']
    df_processed['hasLift'] = df_processed['hasLift'].fillna(mode_hasLift).astype(int)
    df_processed['status'] = df_processed['status'].fillna('Unknown')
    conds = [(df_processed['hasParking']==1)&(df_processed['isParkingIncludedInPrice']==1), (df_processed['hasParking']==1)&(df_processed['isParkingIncludedInPrice']==0), (df_processed['hasParking']==0)]
    choices = ['Included','Optional','None']
    df_processed['parking_status'] = np.select(conds, choices, default='Unknown')
    df_processed.drop(['hasParking','isParkingIncludedInPrice'], axis=1, inplace=True)
    numeric_cols_to_impute = ['size','rooms','bathrooms','latitude','longitude']
    for c in numeric_cols_to_impute:
        df_processed[f'isna_{c}'] = df_processed[c].isna().astype(int)
        median_val = imputation_values[f'median_{c}']
        df_processed[c] = df_processed[c].fillna(median_val)

    # --- 2. Ingeniería de Características Espaciales ---
    print("Generando características espaciales...")
    # (Código igual que antes...)
    coords = list(zip(df_processed['latitude'], df_processed['longitude']))
    for name, loc in pois.items():
        df_processed[f'DistKm_{name}'] = [geodesic(loc, xy).km for xy in coords]
    geo_scaled = scaler_geo.transform(df_processed[['latitude','longitude']])
    df_processed['geo_cluster'] = kmeans_geo.predict(geo_scaled)
    pca1 = pca_geo.transform(geo_scaled)
    df_processed['geo_pca1'] = pca1.flatten()

    # Calcular precio medio del cluster y llenar NaNs
    df_processed['cluster_avg_logprice'] = df_processed['geo_cluster'].map(cluster_avg_logprice_map)
    if isinstance(cluster_avg_logprice_map, (dict, pd.Series)):
        map_values = list(cluster_avg_logprice_map.values()) if isinstance(cluster_avg_logprice_map, dict) else cluster_avg_logprice_map.tolist()
        global_avg_logprice = np.mean(map_values) if map_values else np.log1p(300000)
    else:
        global_avg_logprice = np.log1p(300000)
    # Usar sintaxis recomendada para fillna para evitar FutureWarning
    df_processed['cluster_avg_logprice'] = df_processed['cluster_avg_logprice'].fillna(global_avg_logprice)


    # --- 3. Codificación Categórica (Target Encoding) ---
    print("Aplicando Target Encoding...")
    df_processed[te_cols] = te.transform(df_processed[te_cols]) # type: ignore

    # --- 4. Características No Lineales (Splines y Polinomios) ---
    print("Generando características no lineales...")
    # (Código igual que antes...)
    spline_feats = spline.transform(df_processed[spline_input_cols]) # type: ignore
    spline_names = spline.get_feature_names_out(spline_input_cols) # type: ignore
    df_splines = pd.DataFrame(spline_feats, columns=spline_names, index=df_processed.index)
    df_processed = pd.concat([df_processed, df_splines], axis=1)
    X_poly = poly.transform(df_processed[poly_base_cols]) # type: ignore
    poly_names = poly.get_feature_names_out(poly_base_cols) # type: ignore
    df_poly = pd.DataFrame(X_poly, columns=poly_names, index=df_processed.index)
    new_ints = [c for c in poly_names if ' ' in c]
    df_processed = pd.concat([df_processed, df_poly[new_ints]], axis=1)


    # --- 5. Asegurar Orden y Tipos de Columnas + Escalado Final ---
    print("Ordenando columnas y aplicando escalado final...")
    # (Código igual que antes...)
    for col in feature_order:
        if col not in df_processed.columns:
            df_processed[col] = 0
    df_processed = df_processed[feature_order] # type: ignore
    cols_to_scale_present = [col for col in scaled_num_cols if col in df_processed.columns] # type: ignore
    if cols_to_scale_present:
        for col in cols_to_scale_present:
             df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
        df_processed[cols_to_scale_present] = df_processed[cols_to_scale_present].fillna(0) # Llenar posibles NaNs de conversión
        df_processed[cols_to_scale_present] = scaler_final.transform(df_processed[cols_to_scale_present]) # type: ignore
    else:
        print("Advertencia: No se encontraron columnas para escalar.")

    print("--- Preprocesamiento Completado ---")

    # --- 6. Predicción ---
    print("Realizando predicción...")
    y_pred_log = model.predict(df_processed) # type: ignore
    y_pred_orig = np.expm1(y_pred_log)

    return y_pred_orig

In [11]:
# BLOQUE 6: Realizar la Predicción y Mostrar Resultado
# ---
# Llama a la función de predicción con los datos de entrada y el pipeline cargado.
# Imprime el resultado formateado.

if pipeline_components is not None and input_df is not None:
    # Realizar la predicción
    precio_predicho = predict_price(input_df, pipeline_components)

    if precio_predicho is not None:
        # Mostrar el resultado (tomando el primer elemento si solo hay una predicción)
        precio_final = precio_predicho[0] if len(precio_predicho) == 1 else precio_predicho
        print("\n============================================")
        print(f"  Precio Predicho para la Vivienda:")
        print(f"  >>> {precio_final:,.2f} € <<<") # Formateado como moneda
        print("============================================")

        # Opcional: Mostrar el DataFrame procesado justo antes de la predicción
        # print("\nDataFrame final antes de la predicción (primeras 5 columnas):")
        # print(df_processed.iloc[:, :5].head().to_markdown())

else:
    print("\nNo se puede realizar la predicción porque el pipeline o los datos de entrada no se cargaron correctamente.")


--- Iniciando Preprocesamiento ---
Aplicando limpieza e imputación...
Generando características espaciales...
Aplicando Target Encoding...
Generando características no lineales...
Ordenando columnas y aplicando escalado final...
--- Preprocesamiento Completado ---
Realizando predicción...

  Precio Predicho para la Vivienda:
  >>> 457,586.69 € <<<


<ipython-input-10-4d3694c18cc1>:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_processed['exterior'] = pd.to_numeric(df_processed['exterior'].replace({'Unknown': np.nan, True: 1, False: 0}), errors='coerce').fillna(mode_exterior).astype(int)
